564a67b5fb - PMR3508-H22018 - Parte 2

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
import category_encoders as ce

In [ ]:
train = pd.read_csv("../input/train.csv",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

In [ ]:
train.shape

In [ ]:
train.head()

"Id" column is redundant

In [ ]:
train.pop("Id")

Using "Target Encoding" for pre-processing

In [ ]:
encoder = ce.TargetEncoder(handle_unknown="ignore")

In [ ]:
encoder.fit(train, train.Target)

In [ ]:
train = encoder.transform(train, train["Target"])

In [ ]:
train.head()

Using Mean Imputer on NaNs

In [ ]:
imputer = sklearn.preprocessing.Imputer(strategy = "mean")

In [ ]:
imputer.fit(train)

In [ ]:
trainImputed = train.copy()
trainImputed = pd.DataFrame(imputer.transform(train))
trainImputed.columns = train.columns
trainImputed.index = train.index

In [ ]:
trainImputed.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

Fitting k

In [ ]:
Xtrain = trainImputed.loc[:, trainImputed.columns != "Target"]
Ytrain = trainImputed["Target"]

In [ ]:
list = []

i=10
while (i <= 600):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(Xtrain,Ytrain)
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=7)
    accuracy = sum(scores)/len(scores)
    list.append([i,accuracy])
    i += 10

Plot of accuracy vs k

In [ ]:
pd.DataFrame(list, columns=["k","accuracy"]).plot(x="k",y="accuracy",style="")

In [ ]:
pd.DataFrame(list, columns=["k","accuracy"])

In [ ]:
pd.DataFrame(list, columns=["k","accuracy"])["accuracy"].max()
#Best fit: K = 300

Applying the classifier on test dataset

In [ ]:
test = pd.read_csv("../input/test.csv",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")
test.pop("Id")

In [ ]:
test.head()

In [ ]:
test["Target"] = "0"

test = encoder.transform(test)

testImputed = test.copy()
testImputed = pd.DataFrame(imputer.transform(testImputed))
testImputed.columns = test.columns
testImputed.index = test.index

testImputed["Target"] = ""

In [ ]:
testImputed.head()

In [ ]:
Xtest = testImputed.loc[:, testImputed.columns != "Target"]
Ytest = testImputed["Target"]

knn = KNeighborsClassifier(n_neighbors=300)
knn.fit(Xtrain,Ytrain)
Ytest = knn.predict(Xtest)

In [ ]:
Ytest

In [ ]:
test = pd.read_csv("../input/test.csv",
        sep=r'\s*,\s*',
        engine='python',
        na_values="?")

Evaluation = pd.DataFrame(test["Id"])
Evaluation["Target"] = Ytest

In [ ]:
Evaluation

In [ ]:
Evaluation.to_csv("Evaluation.csv", index=False)